# Veriseti İşleme

In [1]:
import pandas as pd
dataset_path = ("/kaggle/input/turkish-tweet-dataset/TurkishTweets.xlsx")
df = pd.read_excel(dataset_path)
print(df)

                                                  Tweet  Etiket
0     Allah'ım çıldıracağım. Yemin ederim çıldıracağ...  kızgın
1     site ne zaman çalıştıda ürün stokları bitti di...  kızgın
2       Sebebi neydi ki diye bağıracağım şimdi az kaldı  kızgın
3     Black friday ne amk bu yıl çıkardınız bu bokud...  kızgın
4     kısa kısa cevaplar vericeksen niye yazıyorsun ...  kızgın
...                                                 ...     ...
3995                    Aynı şeyleri yapmaktan sıkıldım   üzgün
3996  Olacağı varsa oluyor ya hani, biteceği varsa d...   üzgün
3997    Çok şey istiyorum ama sonu hep fiyasko MUTSUZUM   üzgün
3998  Garip bir şekilde canım çok yanıyor moralim bozuk   üzgün
3999  Üzgünüm birazdan takipten çıkarıp hesabımı giz...   üzgün

[4000 rows x 2 columns]


In [2]:
df = df.rename(columns={"Tweet": "text", "Etiket": "label"})
print(df)

                                                   text   label
0     Allah'ım çıldıracağım. Yemin ederim çıldıracağ...  kızgın
1     site ne zaman çalıştıda ürün stokları bitti di...  kızgın
2       Sebebi neydi ki diye bağıracağım şimdi az kaldı  kızgın
3     Black friday ne amk bu yıl çıkardınız bu bokud...  kızgın
4     kısa kısa cevaplar vericeksen niye yazıyorsun ...  kızgın
...                                                 ...     ...
3995                    Aynı şeyleri yapmaktan sıkıldım   üzgün
3996  Olacağı varsa oluyor ya hani, biteceği varsa d...   üzgün
3997    Çok şey istiyorum ama sonu hep fiyasko MUTSUZUM   üzgün
3998  Garip bir şekilde canım çok yanıyor moralim bozuk   üzgün
3999  Üzgünüm birazdan takipten çıkarıp hesabımı giz...   üzgün

[4000 rows x 2 columns]


In [ ]:
#df = df[["label", "text"]]
#print(df)

In [3]:
df["label"].unique()

array(['kızgın', 'korku', 'mutlu', 'surpriz', 'üzgün'], dtype=object)

In [4]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
print(df)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


                                                   text  label
0     Allah'ım çıldıracağım. Yemin ederim çıldıracağ...      1
1     site ne zaman çalıştıda ürün stokları bitti di...      1
2       Sebebi neydi ki diye bağıracağım şimdi az kaldı      1
3     Black friday ne amk bu yıl çıkardınız bu bokud...      1
4     kısa kısa cevaplar vericeksen niye yazıyorsun ...      1
...                                                 ...    ...
3995                    Aynı şeyleri yapmaktan sıkıldım      4
3996  Olacağı varsa oluyor ya hani, biteceği varsa d...      4
3997    Çok şey istiyorum ama sonu hep fiyasko MUTSUZUM      4
3998  Garip bir şekilde canım çok yanıyor moralim bozuk      4
3999  Üzgünüm birazdan takipten çıkarıp hesabımı giz...      4

[4000 rows x 2 columns]


In [5]:
df["label"].unique()

array([1, 0, 2, 3, 4])

In [6]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.5, random_state=42, shuffle=True, stratify=df['label'])

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

splits = {
    "train": Dataset.from_pandas(train_data),
    "test": Dataset.from_pandas(test_data)
}

tweets = DatasetDict(splits)

print(tweets)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [7]:
tweets['train'][0]

{'text': 'yıldım, yoruldum, tükendim, bunaldım, bıktım, bezdim, sıkıldım, tahammülsüzleştim, bunların tüm sebebi VİZELER',
 'label': 4}

In [8]:
tweets['test'][:3]

{'text': ['Kış gelince hemen tutacak el ararsınız, ben yorgan yastığımla mutluyum',
  'Hala konuşmaları dinleyebiliyor olmana hayret ettim :)',
  "Babam faturamı ödemiş Allah'ım çok mutluyum şu an yoksa bana girecekti kol gibi "],
 'label': [2, 3, 2]}

In [9]:
tweets['train'] = tweets['train'].shuffle(seed=42)
tweets['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [10]:
tweets_train_validation = tweets['train'].train_test_split(train_size=0.8)
tweets_train_validation

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [11]:
tweets_train_validation['validation'] = tweets_train_validation.pop('test')
tweets_train_validation

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [12]:
tweets.update(tweets_train_validation)
tweets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [13]:
tweets['test'] = tweets['test'].shuffle(seed=42).select(range(400))
tweets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [14]:
import pandas as pd
tweets.set_format('pandas')
dd = tweets['train'][:]
dd.label.value_counts()

label
1    326
0    324
3    320
2    317
4    313
Name: count, dtype: int64

In [15]:
print(dd.text.str.len() < 200)

0       True
1       True
2       True
3       True
4       True
        ... 
1595    True
1596    True
1597    True
1598    True
1599    True
Name: text, Length: 1600, dtype: bool


In [16]:
tweets.reset_format()

In [17]:
print(tweets)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})


# Tokenizer

In [18]:
from transformers import AutoTokenizer

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

tweets_encoded = tweets.map(tokenize_function, batched=True, batch_size=None)
tweets_encoded

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 400
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 400
    })
})

In [19]:
print(tweets_encoded['train'][0])

{'text': 'vallahi delireceğim nereye düştük böyle amk ya', 'label': 1, 'input_ids': [101, 191, 5727, 22332, 3687, 5817, 2093, 28223, 4060, 24928, 12210, 1162, 173, 17176, 18272, 1204, 17176, 1377, 171, 19593, 12415, 1821, 1377, 11078, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [20]:
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 5
model = (AutoModelForSequenceClassification
         .from_pretrained(checkpoint, num_labels=num_labels)
         .to(device))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from datasets import DatasetDict

tiny_tweets = DatasetDict()
tiny_tweets['train'] = tweets['train'].shuffle(seed=42).select(range(800))
tiny_tweets['validation'] = tweets['validation'].shuffle(seed=1).select(range(200))
tiny_tweets['test'] = tweets['test'].shuffle(seed=42).select(range(200))

tiny_tweets_encoded = tiny_tweets.map(tokenize_function, batched=True, batch_size=None)
tiny_tweets_encoded

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 800
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 200
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 200
    })
})

# Training

In [23]:
from sklearn.metrics import accuracy_score
def get_accuracy(preds):
  predictions = preds.predictions.argmax(axis=-1)
  labels = preds.label_ids
  accuracy = accuracy_score(preds.label_ids, preds.predictions.argmax(axis=-1))
  return {'accuracy': accuracy}

In [25]:
from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(tiny_tweets_encoded["train"]) // batch_size
model_name = f"{checkpoint}-finetuned-tiny_tweets"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=8,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  log_level="error",
                                  optim='adamw_torch'
                                  )

In [26]:
from transformers import Trainer

torch.cuda.empty_cache()

trainer = Trainer(model=model, 
                  compute_metrics=get_accuracy,
                  args=training_args, 
                  train_dataset=tiny_tweets_encoded["train"],
                  eval_dataset=tiny_tweets_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train();

wandb: Currently logged in as: ozdamarsevval. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,1.608800,1.562275,0.240000
2,1.466600,1.306276,0.535000
3,0.880900,0.732788,0.775000
4,0.359900,0.580715,0.825000
5,0.117300,0.484433,0.870000
6,0.036000,0.528885,0.870000
7,0.018800,0.568247,0.875000
8,0.009200,0.567619,0.875000


In [27]:
trainer.evaluate()

{'eval_loss': 0.567618727684021,
 'eval_accuracy': 0.875,
 'eval_runtime': 0.6738,
 'eval_samples_per_second': 296.837,
 'eval_steps_per_second': 37.105,
 'epoch': 8.0}

In [28]:
preds = trainer.predict(tiny_tweets_encoded['test'])
preds

PredictionOutput(predictions=array([[-1.5352893 , -0.8565649 , -1.4737207 , -2.476218  ,  5.547922  ],
       [-0.99575275, -1.5866103 , -1.248821  ,  5.753561  , -1.7465733 ],
       [-1.1056635 , -1.5080357 , -1.3480211 ,  5.769936  , -1.6463054 ],
       [-1.0536321 ,  5.6121383 , -1.9228073 , -0.22234333, -2.092575  ],
       [-1.1893988 ,  6.0700808 , -1.944446  , -1.2142804 , -1.394316  ],
       [-1.598225  ,  4.875544  , -2.511293  , -0.24189244, -0.59206104],
       [-1.2268834 , -1.3932312 , -1.1972059 ,  5.7116165 , -1.9378648 ],
       [-1.2872946 , -1.1613939 , -1.5282003 , -2.3549552 ,  5.466388  ],
       [-0.9519324 , -2.2044961 , -0.6535998 ,  5.5834165 , -1.8240751 ],
       [ 6.0286584 , -0.8446549 , -1.5422621 , -1.5229012 , -1.2533098 ],
       [-1.6128567 , -2.0768418 , -1.759014  , -0.6793413 ,  5.451552  ],
       [-2.8996484 , -1.2030851 ,  3.79742   , -1.7893398 , -0.0113744 ],
       [ 0.62831175, -0.15363598, -0.83654267,  2.661598  , -2.3373718 ],
       [-

In [29]:
preds.predictions.shape # 200 veri üzerinde 5 sınıf test edildi

(200, 5)

In [30]:
preds.predictions.argmax(axis=-1) # text üzerinde tahmin edilen label değerleri

array([4, 3, 3, 1, 1, 1, 3, 4, 3, 0, 4, 2, 3, 3, 0, 1, 1, 2, 1, 1, 4, 0,
       2, 3, 2, 0, 4, 3, 2, 2, 2, 0, 0, 3, 3, 0, 0, 3, 2, 2, 2, 2, 0, 4,
       3, 2, 4, 4, 4, 2, 3, 1, 4, 2, 1, 4, 3, 4, 4, 3, 1, 4, 3, 2, 0, 1,
       1, 2, 2, 1, 4, 3, 1, 2, 2, 4, 3, 0, 3, 1, 2, 3, 1, 1, 4, 2, 3, 3,
       3, 3, 4, 3, 4, 4, 4, 1, 4, 1, 4, 0, 3, 3, 1, 0, 2, 4, 4, 4, 3, 3,
       2, 2, 4, 2, 4, 1, 4, 3, 3, 0, 3, 0, 0, 4, 0, 4, 1, 1, 3, 2, 0, 4,
       1, 3, 4, 4, 0, 1, 2, 2, 3, 4, 2, 1, 1, 3, 3, 3, 0, 4, 1, 2, 0, 4,
       3, 3, 3, 2, 4, 1, 2, 2, 2, 2, 3, 4, 1, 0, 0, 0, 4, 4, 0, 4, 4, 2,
       0, 3, 0, 1, 2, 4, 1, 4, 0, 2, 3, 0, 1, 2, 0, 1, 3, 2, 3, 4, 4, 1,
       3, 1])

In [31]:
preds.label_ids # text verilerinin gerçek label değerleri

array([4, 3, 3, 1, 1, 1, 3, 4, 3, 0, 4, 3, 3, 3, 0, 1, 1, 2, 4, 1, 4, 0,
       2, 3, 2, 0, 4, 3, 2, 2, 4, 0, 0, 3, 3, 0, 0, 3, 2, 2, 2, 2, 0, 3,
       3, 2, 4, 1, 4, 2, 3, 1, 4, 2, 1, 4, 3, 4, 2, 3, 1, 4, 3, 2, 0, 1,
       1, 2, 2, 1, 4, 3, 1, 2, 2, 4, 3, 0, 3, 1, 2, 3, 1, 1, 4, 2, 3, 3,
       3, 3, 4, 3, 4, 4, 4, 1, 4, 1, 2, 0, 1, 3, 1, 0, 2, 0, 4, 4, 1, 2,
       2, 2, 4, 2, 4, 1, 2, 3, 3, 0, 3, 0, 0, 4, 0, 4, 1, 1, 3, 2, 0, 4,
       1, 3, 4, 4, 0, 1, 2, 2, 1, 4, 2, 1, 0, 3, 3, 3, 0, 4, 1, 2, 0, 4,
       3, 3, 3, 2, 4, 1, 2, 2, 2, 2, 3, 3, 1, 0, 0, 0, 4, 4, 0, 4, 4, 2,
       0, 3, 0, 1, 2, 1, 1, 4, 0, 2, 3, 0, 1, 2, 0, 0, 3, 2, 3, 4, 4, 1,
       3, 1])

In [32]:
from sklearn.metrics import accuracy_score

accuracy_score(preds.label_ids, preds.predictions.argmax(axis=-1))

0.915

# Training - 2

In [33]:
batch_size = 16
logging_steps = len(tweets_encoded["train"]) // batch_size
model_name = f"{checkpoint}-finetuned"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=10,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  log_level="error",
                                  optim='adamw_torch'
                                  )

In [34]:
from transformers import Trainer

torch.cuda.empty_cache()

trainer = Trainer(model=model, 
                  args=training_args, 
                  compute_metrics=get_accuracy,
                  train_dataset=tweets_encoded["train"],
                  eval_dataset=tweets_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.209600,0.253532,0.937500
2,0.083500,0.293057,0.942500
3,0.035800,0.211152,0.952500
4,0.012400,0.239954,0.947500
5,0.007100,0.224096,0.955000
6,0.002400,0.234240,0.957500
7,0.003500,0.234463,0.960000
8,0.000500,0.227808,0.957500
9,0.005900,0.224327,0.960000
10,0.000300,0.239635,0.957500


TrainOutput(global_step=1000, training_loss=0.03610659484937787, metrics={'train_runtime': 164.639, 'train_samples_per_second': 97.182, 'train_steps_per_second': 6.074, 'total_flos': 674240240064000.0, 'train_loss': 0.03610659484937787, 'epoch': 10.0})

In [35]:
trainer.evaluate()

{'eval_loss': 0.23963455855846405,
 'eval_accuracy': 0.9575,
 'eval_runtime': 1.1576,
 'eval_samples_per_second': 345.529,
 'eval_steps_per_second': 21.596,
 'epoch': 10.0}

In [36]:
trainer.save_model()

In [37]:
model_name

'bert-base-cased-finetuned'

In [38]:
from transformers import pipeline
classifier = pipeline('text-classification', model=model_name)

In [ ]:
#'kızgın' = 1, 'korku' = 0, 'mutlu' = 2, 'surpriz' = 3, 'üzgün' = 4

In [39]:
classifier('kazadan sonra aracın aldığı hasar çok fazlaydı')

[{'label': 'LABEL_4', 'score': 0.8361601829528809}]

In [45]:
classifier('sana bir sürprizim var')

[{'label': 'LABEL_3', 'score': 0.9998212456703186}]

In [46]:
pip install transformers --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/ad/67/b4d6a51dcaf988cb45b31e26c6e33fb169fe34ba5fb168b086309bd7c028/transformers-4.37.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 76.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstalled transformers-4.36.2
Note: you may need to restart the kernel to use updated packages.


In [47]:
pip install huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [48]:
from huggingface_hub import notebook_login
notebook_login()

In [49]:
trainer.push_to_hub()

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

events.out.tfevents.1706260083.677d0e3c3311.251.2:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.28k [00:00<?, ?B/s]

events.out.tfevents.1706260261.677d0e3c3311.251.3:   0%|          | 0.00/411 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sevvalozdamar/bert-base-cased-finetuned/commit/abeb2037dc9b218349f2cee3729833c2b2515f01', commit_message='End of training', commit_description='', oid='abeb2037dc9b218349f2cee3729833c2b2515f01', pr_url=None, pr_revision=None, pr_num=None)